In [1]:
import cv2
import urllib
import numpy as np
from random import randrange, uniform

from sklearn import cluster, datasets

#check version
print("OpenCv version: ", cv2.__version__)

OpenCv version:  4.5.3


In [2]:
LINE_WIDTH_SPEC = 3

# Parameters for Hough line detection
rho = 1.0
theta = np.pi/180
threshold = 20
min_line_length = 10
max_line_gap = 100

# Parameters for binarize threshold and Canny Line detection
bi_thd = 30
bi_max = 255

#img_path = "D:/Research/opencv/image/dora/b.jpg"
img_path = "D:/Research/opencv/image/dora/stitch_test.jpg"
#img_path = "D:/Research/opencv/image/adas/traffic.jpg"

nor_color = (0, 255, 255)
warn_color = (0,0,255)
font = cv2.FONT_HERSHEY_SIMPLEX
color = (0, 255, 255)

In [3]:
def displayResizedImg(name, img, factor):
    imgDis = cv2.resize(img, None, fx=factor, fy=factor, interpolation=cv2.INTER_AREA)
    cv2.imshow(name, imgDis)
    return imgDis

In [4]:
def findLimitValue(arr):
    #print("min: {} max: {}".format(min(arr), max(arr)))
    return [min(arr), max(arr), max(arr)-min(arr)]

In [5]:
def detectlineWidth(lines, groups):
    data = np.array(lines)
    kmeans_fit = cluster.KMeans(n_clusters=groups).fit(data.reshape(-1,1))
    cluster_labels = kmeans_fit.labels_
    
    temp = [[0 for x in range(2)] for x in range(len(cluster_labels))]
    temp = np.array(temp)
    for i in range(len(cluster_labels)):
        temp[i,0] = cluster_labels[i]
        temp[i,1] = data[i]
        
    #Sort by y position
    temp = sorted(temp,key=lambda temp : temp [1])
    
    idx = temp[0][0]
    test = []
    final_list = []
    for i in range(len(temp)):
        #print("Index {}, array value {} {}".format(idx, temp[i][0], temp[i][1]))
        if idx != temp[i][0]:
            idx = temp[i][0]
            final_list.append(findLimitValue(test))
            test.clear()
            test.append(temp[i][1])
        else:
            test.append(temp[i][1])
    #final_list += findLimitValue(test)
    final_list.append(findLimitValue(test))
    
    #slices a[start:end]
    #direct access a[x][y]
    
    return final_list

In [6]:
#read from file
img = cv2.imread(img_path)

In [7]:
#Convert image to gray
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

In [8]:
#Select and Crop image
showCrosshair = False
fromCenter = False
# Select ROI
rect = cv2.selectROI("select roi", gray, showCrosshair, fromCenter)
x, y, w, h = rect

# Crop image
imgGrayCrop = gray[y : y+h, x:x+w]
imCrop = img[y : y+h, x:x+w]

In [9]:
#Detect line Canny
#imCanny = cv2.blur(imgGrayCrop, (3,3))
#edges = cv2.Canny(imCanny, 90.0, 180.0)

In [10]:
# detect Gradient
imgGrayCrop = cv2.blur(imgGrayCrop, (3,3))
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3,3))
gradient = cv2.morphologyEx(imgGrayCrop, cv2.MORPH_GRADIENT, kernel, iterations=2)

gradient = cv2.morphologyEx(gradient, cv2.MORPH_OPEN, kernel, iterations=2)
gradient = cv2.erode(gradient, kernel, iterations = 1)
ret, gradient = cv2.threshold(gradient, bi_thd, bi_max, cv2.THRESH_BINARY)
gradient = cv2.erode(gradient, kernel, iterations = 1)

gradient = displayResizedImg('Gradient after binarization*5', gradient, 5)

# detect edges
edges = cv2.Canny(gradient, bi_thd, 255.0)
displayResizedImg('Edges', edges, 1)

# draw countours 
contours, hierarchy = cv2.findContours(gradient, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
imCropC = cv2.resize(imCrop, None, fx=5, fy=5, interpolation=cv2.INTER_AREA)
imCropC = cv2.drawContours(imCropC.copy(),contours,-1,(0,255,255),3)
displayResizedImg('Contour', imCropC, 1)
print(len(contours))

lines = cv2.HoughLinesP(edges,rho,theta,threshold,np.array([]),min_line_length, max_line_gap)
imCropA = cv2.resize(imCrop, None, fx=5, fy=5, interpolation=cv2.INTER_AREA)
displayResizedImg('hough', imCropA, 1)

y_list = []

if lines is not None:
    for line in lines:
            x1,y1,x2,y2 = line.reshape(4)
            y_list.append(y1)
            #cv2.line(imCropA, (x1,y1), (x2,y2), color, 1)
            #cv2.putText(imCropA,'{}'.format(int(y1)), (x1, y1), font, 1, color, 1, cv2.LINE_AA)

    #print(y_list)
    #Group data
    lines_result = detectlineWidth(np.array(y_list), len(contours))
    #print(lines_result)
    
    w = imCropA.shape[1]
    
    # draw result on image
    for result in lines_result:
        #print("{} {} {}".format(result[0], result[1], result[2]))
        lw = float(result[2])/5
        print("{}".format(lw))
        if lw > LINE_WIDTH_SPEC:
            color = warn_color
        else:
            color = nor_color
        cv2.line(imCropA, (0,result[0]), (w,result[0]), color, 1)
        cv2.line(imCropA, (0,result[1]), (w,result[1]), color, 1)
        cv2.line(imCropA, (100,result[0]), (100,result[1]), color, 1)
        cv2.putText(imCropA,'{}'.format(lw), (110, int((result[1]+result[0])/2)), font, 1, color, 1, cv2.LINE_AA)
        
    

10
3.0
3.0
2.0
2.0
3.0
3.0
3.0
3.0
3.0
4.0


In [11]:
#Show images
cv2.imshow('result', imCropA)
#cv2.imshow('edge', edges)

#cv2.imshow('Original image', img)
#displayResizedImg('ROI', imgGrayCrop, 5)


In [12]:
#wait keydown and then close all windows
cv2.waitKey()
cv2.destroyAllWindows()